<a href="https://colab.research.google.com/github/PhaneendraYedidi/MachineHackHackathon/blob/main/UnitPricesPrediction2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV 

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")


In [ ]:
#Mounting the Google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Reading the train and test data from drive
pathExt = 'drive/MyDrive/HackathonData/'
trainData = pd.read_csv(pathExt + "Train.csv")
testData = pd.read_csv(pathExt + "Test.csv")

Business Constraints:

*   Multivariate Regression
*   Big dataset, underfitting vs overfitting
*   Optimizing RMSE to generalize well on unseen data






#Data Cleaning

In [ ]:
trainData.head()
#trainData.value_counts()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [ ]:
testData.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
0,3139,1709,1079,3,2011-02-22 15:22:00,16161.0,35
1,617,510,3457,1,2010-12-08 12:46:00,17341.0,35
2,14653,604,694,36,2011-10-25 13:53:00,15158.0,35
3,8634,1478,3473,2,2011-06-27 12:38:00,16033.0,35
4,15546,3216,871,1,2011-11-06 16:14:00,15351.0,35


In [ ]:
trainData.describe()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
count,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000
mean,9955.394083,1573.386807,2023.955573,12.028468,3.451216,15288.224278,32.823846
std,5551.519138,843.604991,1089.812078,295.111588,78.399541,1714.336529,6.743602
min,0.000000,0.000000,0.000000,-80995.000000,0.000000,12346.000000,0.000000
25%,5069.000000,939.000000,1141.000000,2.000000,1.250000,13953.000000,35.000000
50%,10310.000000,1521.000000,1987.000000,5.000000,1.950000,15152.000000,35.000000
75%,14657.000000,2106.000000,2945.000000,12.000000,3.750000,16794.000000,35.000000
max,22188.000000,3683.000000,3895.000000,80995.000000,38970.000000,18287.000000,36.000000


In [ ]:
#Min value of Quantity is negative which is not realistic so removing the values
print('Shape of train data',trainData.shape)
trainData = trainData[trainData['Quantity']>0]
print('Shape of train data',trainData.shape)

Shape of train data (284780, 8)
Shape of train data (278613, 8)


In [ ]:
trainData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278613 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    278613 non-null  int64  
 1   StockCode    278613 non-null  int64  
 2   Description  278613 non-null  int64  
 3   Quantity     278613 non-null  int64  
 4   InvoiceDate  278613 non-null  object 
 5   UnitPrice    278613 non-null  float64
 6   CustomerID   278613 non-null  float64
 7   Country      278613 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 19.1+ MB


No Null values in the Training data

In [ ]:
for i in np.arange(10,101,10):
  p = np.percentile(trainData.Quantity, i)
  print('Percentile ', str(i), "is ", str(p))

Percentile  10 is  1.0
Percentile  20 is  2.0
Percentile  30 is  2.0
Percentile  40 is  4.0
Percentile  50 is  6.0
Percentile  60 is  8.0
Percentile  70 is  12.0
Percentile  80 is  12.0
Percentile  90 is  24.0
Percentile  100 is  80995.0


In [ ]:
for i in np.arange(90,101,1):
  p = np.percentile(trainData.Quantity, i)
  print('Percentile ', str(i), "is ", str(p))

Percentile  90 is  24.0
Percentile  91 is  24.0
Percentile  92 is  24.0
Percentile  93 is  25.0
Percentile  94 is  32.0
Percentile  95 is  36.0
Percentile  96 is  48.0
Percentile  97 is  50.0
Percentile  98 is  96.0
Percentile  99 is  120.0
Percentile  100 is  80995.0


In [ ]:
for i in np.arange(99,100.1,.1):
  p = np.percentile(trainData.Quantity, i)
  print('Percentile ', str(i), "is ", str(p))

Percentile  99.0 is  120.0
Percentile  99.1 is  144.0
Percentile  99.19999999999999 is  144.0
Percentile  99.29999999999998 is  144.0
Percentile  99.39999999999998 is  192.0
Percentile  99.49999999999997 is  200.0
Percentile  99.59999999999997 is  216.0
Percentile  99.69999999999996 is  256.0
Percentile  99.79999999999995 is  320.0
Percentile  99.89999999999995 is  500.0
Percentile  99.99999999999994 is  80994.99999881606


In [ ]:
#There is a big change in percentile of quantity from 99.89. This can be thought of as outlier
print('Shape of train data',trainData.shape)
#trainData = trainData[trainData['Quantity']>=500]
trainData = trainData[trainData.Quantity < np.percentile(trainData.Quantity,99.9)]
print('Shape of train data',trainData.shape)

Shape of train data (278613, 8)
Shape of train data (278323, 8)


In [ ]:
corr_matrix = trainData.corr(method='pearson')
corr_matrix

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
InvoiceNo,1.000000,0.084850,0.026582,-0.014877,-0.005964,-0.000570,0.010956
StockCode,0.084850,1.000000,-0.012827,-0.003821,0.043011,0.002713,0.007056
Description,0.026582,-0.012827,1.000000,0.006607,-0.003094,-0.005833,-0.014649
Quantity,-0.014877,-0.003821,0.006607,1.000000,-0.036476,-0.047285,-0.099469
UnitPrice,-0.005964,0.043011,-0.003094,-0.036476,1.000000,-0.017241,-0.024367
CustomerID,-0.000570,0.002713,-0.005833,-0.047285,-0.017241,1.000000,0.387694
Country,0.010956,0.007056,-0.014649,-0.099469,-0.024367,0.387694,1.000000


In [ ]:
trainData['Year'] = pd.to_datetime(trainData.InvoiceDate).dt.year
testData['Year'] = pd.to_datetime(testData.InvoiceDate).dt.year
trainData['Week'] = pd.to_datetime(trainData.InvoiceDate).dt.week
testData['Week'] = pd.to_datetime(testData.InvoiceDate).dt.week
trainData['day'] = pd.to_datetime(trainData.InvoiceDate).dt.day
testData['day'] = pd.to_datetime(testData.InvoiceDate).dt.day
trainData['Month'] = pd.to_datetime(trainData.InvoiceDate).dt.month
testData['Month'] = pd.to_datetime(testData.InvoiceDate).dt.month
trainData['Weekday'] = pd.to_datetime(trainData.InvoiceDate).dt.dayofweek
testData['Weekday'] = pd.to_datetime(testData.InvoiceDate).dt.dayofweek

trainData['WeekEnd'] = (pd.to_datetime(trainData.InvoiceDate).dt.dayofweek == 6).astype(float)
testData['WeekEnd'] = (pd.to_datetime(testData.InvoiceDate).dt.dayofweek == 6).astype(float)

festive={9:1,10:1,11:1,12:1}
trainData["isfestive"]=trainData.Month.map(festive)
trainData["isfestive"]=trainData["isfestive"].fillna(0)
trainData["isfestive"]=trainData["isfestive"].astype('int32')
testData["isfestive"]=testData.Month.map(festive)
testData["isfestive"]=testData["isfestive"].fillna(0)
testData["isfestive"]=testData["isfestive"].astype('int32')
trainData['Qtr'] = pd.to_datetime(trainData.InvoiceDate).dt.quarter
testData['Qtr'] = pd.to_datetime(testData.InvoiceDate).dt.quarter

In [ ]:
#feature_cols = ["StockCode", "Quantity" ]
#feature_cols = ["StockCode","Description", 	"Quantity",	"CustomerID",	"Country"]
x = trainData.drop(['InvoiceDate','InvoiceNo', 'UnitPrice'],1, inplace= False)#trainData[feature_cols]
y = trainData["UnitPrice"]

In [ ]:
from sklearn.preprocessing import PowerTransformer
y = y.values.reshape(-1,1)
# power transform the raw data
power = PowerTransformer(method='yeo-johnson', standardize=True)
y = power.fit_transform(y)

In [ ]:
# Split data into train and test format
#x_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size=0.2, random_state=None)

In [ ]:
!pip install featuretools

In [ ]:
import featuretools as ft
import featuretools.variable_types as vtypes

df2=x
es = ft.EntitySet(id = 'hackathons')
#removing categories below or else it will just ohe them which is not ideal in many cases

es.entity_from_dataframe(entity_id = 'data', dataframe = df2, make_index = True, index = 'index')
feature_matrix, feature_list = ft.dfs(entityset = es, target_entity = 'data',
                                      agg_primitives=["mean", "max", "min", "std", "skew"],
                                      trans_primitives=["time_since_previous", 'Absolute'],
                                      max_depth=2)
x=feature_matrix

In [ ]:
print(feature_matrix)

        StockCode  Description  ...  ABSOLUTE(day)  ABSOLUTE(isfestive)
index                           ...                                    
0            1583          144  ...              6                    0
1            1300         3682  ...             11                    0
2            2178         1939  ...             20                    1
3            2115         2983  ...             22                    1
4            1210         2886  ...              6                    0
...           ...          ...  ...            ...                  ...
278318       1706         1081  ...              1                    0
278319       2184         2947  ...             13                    1
278320        667         3632  ...             16                    0
278321       1576         3089  ...             11                    1
278322       2603         1156  ...             22                    0

[278323 rows x 26 columns]


In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278323 entries, 0 to 278322
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   StockCode              278323 non-null  int64  
 1   Description            278323 non-null  int64  
 2   Quantity               278323 non-null  int64  
 3   CustomerID             278323 non-null  float64
 4   Country                278323 non-null  int64  
 5   Year                   278323 non-null  int64  
 6   Week                   278323 non-null  int64  
 7   day                    278323 non-null  int64  
 8   Month                  278323 non-null  int64  
 9   Weekday                278323 non-null  int64  
 10  WeekEnd                278323 non-null  float64
 11  isfestive              278323 non-null  int32  
 12  Qtr                    278323 non-null  int64  
 13  ABSOLUTE(Country)      278323 non-null  int64  
 14  ABSOLUTE(CustomerID)   278323 non-nu

In [ ]:
x_test = testData.drop(['InvoiceDate','InvoiceNo'],1, inplace= False)
df1=x_test
es = ft.EntitySet(id = 'hackathonstd')
#removing categories below or else it will just ohe them which is not ideal in many cases

es.entity_from_dataframe(entity_id = 'data', dataframe = df1, make_index = True, index = 'index')
feature_matrix, feature_list = ft.dfs(entityset = es, target_entity = 'data',
                                      agg_primitives=["mean", "max", "min", "std", "skew"],
                                      trans_primitives=["time_since_previous", 'Absolute'],
                                      max_depth=2)
x_test=feature_matrix

#KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Creating the hyperparameter grid  
param_grid = {"n_neighbors": list(range(1,6))} 
  
# Instantiating model 
model = KNeighborsRegressor(weights = 'distance', p=1)
  
# Instantiating RandomizedSearchCV object 
rand_search = RandomizedSearchCV(model, param_grid, cv = 5, scoring='neg_root_mean_squared_error') 
  
rand_search.fit(x, y)
print(rand_search.best_estimator_)
model_best = KNeighborsRegressor(n_neighbors = rand_search.best_estimator_.n_neighbors, weights = 'distance', p=1)
model_best.fit(x, y)

y_pred_train_knn = model_best.predict(x)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y,y_pred_train_knn)))


y_pred_test_knn = model_best.predict(x_test)
print(y_pred_test_knn)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=3, p=1,
                    weights='distance')
RMSE: 0.011596198340195592
[[-0.18217146]
 [-0.51403466]
 [ 0.85903118]
 ...
 [-0.51403466]
 [ 1.07975018]
 [ 0.91310376]]


#Decision Tree Regressor

In [ ]:
'''
from sklearn.tree import DecisionTreeRegressor

# Creating the hyperparameter grid  
param_grid = {"min_samples_split": [ 30, 40,50],
              "max_depth": [ 12, 14, 16]
              } 
  
# Instantiating model 
model = DecisionTreeRegressor()
  
# Instantiating RandomizedSearchCV object 
rand_search = RandomizedSearchCV(model, param_grid, cv = 3, scoring='neg_root_mean_squared_error') 
  
rand_search.fit(x, y)
best_params = rand_search.best_params_
print(best_params)

model_best = DecisionTreeRegressor(**best_params)
model_best.fit(x, y)

y_pred_train_dt = model_best.predict(x)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y,y_pred_train_dt)))

x_test = testData.drop(['InvoiceDate','InvoiceNo'],1, inplace= False)
y_pred_test_dt = model_best.predict(x_test)
print(y_pred_test_dt)
'''

#GBT

In [ ]:
'''
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import randint as sp_randint

param_grid = {'n_estimators':range(20,81,10)          
         }

model = GradientBoostingRegressor(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10)
  
# Instantiating RandomizedSearchCV object 
rand_search = RandomizedSearchCV(model, param_grid, cv = 3, scoring='neg_root_mean_squared_error') 
  
rand_search.fit(x, y.flatten())
best_params = rand_search.best_params_
print(best_params)

model_best = GradientBoostingRegressor(**best_params, learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10)
model_best.fit(x, y)

y_pred_train_dt = model_best.predict(x)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y,y_pred_train_dt)))

x_test = testData.drop(['InvoiceDate','InvoiceNo'],1, inplace= False)
y_pred_test_dt = model_best.predict(x_test)
print(y_pred_test_dt)
'''

#Light GBM

In [ ]:
'''
!pip install lightgbm
from lightgbm import LGBMRegressor
from scipy.stats import randint as sp_randint

param_grid = {'n_estimators': sp_randint(200, 1500),
          'max_depth': sp_randint(2, 15)          
         }

model = LGBMRegressor(subsample=0.9)
  
# Instantiating RandomizedSearchCV object 
rand_search = RandomizedSearchCV(model, param_grid, cv = 3, scoring='neg_root_mean_squared_error') 
  
rand_search.fit(x, y.flatten())
best_params = rand_search.best_params_
print(best_params)


model_best = LGBMRegressor(**best_params, subsample=0.9, random_state=42, n_jobs=-1)
model_best.fit(x, y)

y_pred_train_dt = model_best.predict(x)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y,y_pred_train_dt)))

x_test = testData.drop(['InvoiceDate','InvoiceNo'],1, inplace= False)
y_pred_test_dt = model_best.predict(x_test)
print(y_pred_test_dt)
'''

# MLP model

In [ ]:
'''
import keras as ks
model_in = ks.Input(shape=(x_train.shape[1],), dtype='float32', sparse=True

out = ks.layers.Dense(256, activation='relu')(model_in)
out = ks.layers.Dense(64, activation='relu')(out)
out = ks.layers.Dense(64, activation='relu')(out)
out = ks.layers.Dense(32, activation='relu')(out)
out = ks.layers.Dense(1)(out)
model = ks.Model(model_in, out)

model.compile(loss='mean_squared_error', optimizer=ks.optimizers.Adam(lr=3e-3))
    for i in range(2):
        with timer(f'epoch {i + 1}'):
            model.fit(x=x_train, y=y_train, batch_size=2**(9 + i), epochs=1, verbose=1, validation_data=(x_cv, y_cv))
'''

#Post Processing

In [ ]:

y_test_final_trans = power.inverse_transform(y_pred_test_knn.round(3).reshape(-1, 1))
y_test_final_trans

array([[1.6502337 ],
       [1.25003689],
       [3.95448379],
       ...,
       [1.25003689],
       [4.83256727],
       [4.14961395]])

In [ ]:
np.argwhere(np.isnan(y_test_final_trans))

array([], shape=(0, 2), dtype=int64)

In [ ]:
prediction = pd.DataFrame(np.array(y_test_final_trans.round(2)), columns=['UnitPrice']).to_csv('UnitPricesPrediction.csv')